# CSI 4142 Deliverable 2 - Phase 2
### Group 35
- Yasin Elmi, 300163765
- Oluwatobiloba Ogunbi, 300202843
- Michael Thompson, 300175414

## Intial reading of files into dataframes

Note: may need to adjust the file name and/or location if this notebook is not within the same directory as the data files.

In [108]:
import pandas as pd
import json

df_movies = pd.read_csv('datasets/tmdb_5000_movies.csv', index_col=False)
df_credits = pd.read_csv('datasets/tmdb_5000_movies.csv')

# Transforming of data
The following section is the process of our data being cleaned and transformed

## Dropping columns
Initial dropping of unnecessary columns, according to our first deliverable plan

In [109]:
# Dropping unneccessary columns
df_movies = df_movies.drop('homepage', axis=1)
df_movies = df_movies.drop('id', axis=1)
df_movies = df_movies.drop('keywords', axis=1)
df_movies = df_movies.drop('overview', axis=1)
df_movies = df_movies.drop('status', axis=1)
df_movies = df_movies.drop('tagline', axis=1)

## Cleaning attributes containing JSON objects as String
The 'genre', 'production_companies' and 'production_countries' attributes contain a JSON as a string, which contains a desired key-value pair. A function was created to extract the desired value within the string, and return it as a list.

### TODO
Some of the rows contain an empty value, denoted by the empty square brackets []. Must decide on how to handle this.

In [110]:
# Cleaning the genres, production_companies and production_countries columns from json string

def extract_json_key(string, key):
    '''
    Helper function used to translate string to list of dictionaries, and extract the 'name' key-value pair
    (Str, Str) -> Str
    Preconditions: None
    '''
    clean_list = json.loads(string)
    name = [x[key] for x in clean_list]
    return ', '.join(name)

# Calling the above function to extract the respective 'name' key-value 
df_movies['genres'] = df_movies['genres'].map(lambda x: extract_json_key(x, 'name'))
df_movies['production_companies'] = df_movies['production_companies'].map(lambda x: extract_json_key(x, 'name'))
df_movies['production_countries'] = df_movies['production_countries'].map(lambda x: extract_json_key(x, 'name'))


## Cleaning the language attributes
This section is focused on cleaning the two langauge attribtues, 'original_language' and 'spoken_language'. 

'original_language' is originally written in ISO 639-1 code, which is a 2 letter abbreviation of a word. Using a dictionary found below, the word is swapped for it's verbose display.

'spoken_language' is a little more difficult, where a film can have multiple spoken languages. This is stored in a similar JSON string as above which stores both the ISO 639-1 code and the full written word. Unfortunately, the full written word may utilize a unicode character (i.e. 'French' written in French has an accent, which stores a unicode character). The solution to this is utililizing our previously made dictionary and search it using the provided ISO 639-1 code. A function takes the list of JSON objects, extracts the ISO 639-1 code and translates it, then returns.

### TODO:
- Some of the columns are empty (denoted by an empty set of square brackets []), an example of this is column k row 2650. Need to make decision on how to handle this

In [111]:
# Create dictionary mapping 2 letter language abbrv. to full word
language_two_code_dict = {
    'en': 'English',
    'fr': 'French',
    'ja': 'Japanese',
    'zh': 'Chinese',
    'es': 'Spanish',
    'de': 'German',
    'hi': 'Hindi',
    'ru': 'Russian',
    'ko': 'Korean',
    'te': 'Telugu',
    'cn': 'Chinese',
    'it': 'Italian',
    'nl': 'Dutch',
    'ta': 'Tamil',
    'sv': 'Swedish',
    'th': 'Thai',
    'da': 'Danish',
    'xx': 'No Language',
    'hu': 'Hungarian',
    'cs': 'Czech',
    'pt': 'Portuguese',
    'is': 'Icelandic',
    'tr': 'Turkish',
    'nb': 'Norwegian Bokmal',
    'af': 'Afrikaans',
    'pl': 'Polish',
    'he': 'Hebrew',
    'ar': 'Arabic',
    'vi': 'Vietnamese',
    'ky': 'Kyrgyz',
    'id': 'Indonesian',
    'ro': 'Romanian',
    'fa': 'Persian',
    'no': 'Norwegian',
    'sl': 'Slovenian',
    'ps': 'Pashto',
    'el': 'Greek'
}

def language_lookup(word):
    '''
    Helper function that searches a ISO 639-1 code in the dictionary and returns the verbose word
    String -> String
    '''
    if(word in language_two_code_dict):
        return language_two_code_dict.get(word)
    else:
        return 'NEED TO DOUBLE CHECK MISSING'

df_movies['original_language'] = df_movies['original_language'].map(lambda x: language_lookup(x))


def translate_and_clean(string):
    '''
    Helper function that takes string of 2 code abbreviations, translates and returns in a String
    (Str) -> Str
    '''
    extract = extract_json_key(string, 'iso_639_1').split(',')
    for i in range(len(extract)):
        extract[i] = extract[i].strip()
        extract[i] = language_lookup(extract[i])
    
    return ', '.join(extract)

# Apply function on each row within 'spoken_languages' column
df_movies['spoken_languages'] = df_movies['spoken_languages'].map(lambda x: translate_and_clean(x))

# Staging the data
Below is the staging part of our script

### Cleaning missing/null values
In the section below we will handle all null/missing values within our dataset

In [112]:
print(df_movies.isnull().sum())

budget                  0
genres                  0
original_language       0
original_title          0
popularity              0
production_companies    0
production_countries    0
release_date            1
revenue                 0
runtime                 2
spoken_languages        0
title                   0
vote_average            0
vote_count              0
dtype: int64


runtime has 2 missing values and release_date has 1 missing or null value

In [113]:
df_movies[df_movies.isnull().any(axis=1)]
# should we change these rows manually?

,budget,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count
2656,15000000,Drama,Italian,Chiamatemi Francesco - Il Papa della gente,0.738646,Taodue Film,Italy,2015-12-03,0,NaN,Spanish,Chiamatemi Francesco - Il Papa della gente,7.3,12
4140,2,Documentary,English,"To Be Frank, Sinatra at 100",0.050625,Eyeline Entertainment,United Kingdom,2015-12-12,0,NaN,NEED TO DOUBLE CHECK MISSING,"To Be Frank, Sinatra at 100",0.0,0
4553,0,,English,America Is Still the Place,0.000000,,,NaN,0,0.0,NEED TO DOUBLE CHECK MISSING,America Is Still the Place,0.0,0


In [116]:
# Drop 2 rows - 4553 & 4140 (Too many null/empty values)
# 1037 rows with 'budget' = 0
# 28 rows with 'genre' = ''
# 351 rows with 'production_companies' = ''
# 174 rows with 'production_countries' = ''
# Already dropping row with NaN (4553)
# 35 rows with 'runtime' = 0
# 1427 rows with revenue = 0
# 1 row with popularity = 0 (I think this is safe - unpopular movie?)
# 63 rows with vote_average = 0.0
# 62 rows with vote_count = 0


# 261 rows where original title != title (Thinking foreign films?)


#len(df_movies.loc[df_movies['budget']==0])
#len(df_movies.loc[df_movies['genres']==''])
#len(df_movies.loc[df_movies['original_language']])
#len(df_movies.loc[df_movies['original_title']==''])
#len(df_movies.loc[df_movies['popularity']==0.0])
#len(df_movies.loc[df_movies['production_companies']==''])
#len(df_movies.loc[df_movies['production_countries']==''])
#len(df_movies.loc[df_movies['revenue']==0])
#len(df_movies.loc[df_movies['original_title']==''])
#len(df_movies.loc[df_movies['title']==''])
#len(df_movies.loc[df_movies['runtime']==0])
#df_movies.loc[df_movies['title'] !=df_movies['original_title']]
#len(df_movies.loc[df_movies['vote_average']==0.0])
#len(df_movies.loc[df_movies['vote_count'] < 0])



0